# core

> Main(/all) routines for `ipyfernel`

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from jupyter_client.manager import KernelManager
from IPython.display import display, Image
import base64
from pathlib import Path
from IPython.core.magic import register_line_magic, register_line_cell_magic

## Remote Kernel Backend

In [ ]:
#| export
#| notest
def set_ssh_config(
    port:int,                       # Port number on proxy server (e.g. bore.pub)
    user:str="",                    # Username on remote system.
    alias="remote_server_sshpyk",   # Default alias for `sshpyk`, leave it alone
    proxyname="bore.pub",           # Have tested this with bore
    config_path="~/.ssh/config",    # Shouldn't need to change this.
    ):
    config_path = Path(config_path).expanduser()
    if not config_path.exists(): config_path.touch()
    text = config_path.read_text()
    if f"Host {alias}" not in text: 
        assert user != "", "Must specify username when creating ~/.ssh/config info"
        block = f"""
Host {alias}
    HostName {proxyname}
    Port {port}
    User {user}
    BatchMode yes
    ControlMaster auto
    ControlPath ~/.ssh/sshpyk_%r@%h_%p
    ControlPersist 10m
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
"""
        config_path.write_text(text + block)
    else:
        lines = text.splitlines()
        in_target_block = False
        for i, line in enumerate(lines):
            if line.startswith("Host "):
                in_target_block = (line == f"Host {alias}")
            elif in_target_block and line.strip().startswith("Port "):
                lines[i] = f"    Port {port}"
            elif proxyname and in_target_block and line.strip().startswith("HostName "):
                lines[i] = f"    HostName {proxyname}"
            elif user and in_target_block and line.strip().startswith("User "):
                lines[i] = f"    User {user}"
        config_path.write_text("\n".join(lines) + "\n")
    print(f'{config_path} file updated.') 

In [ ]:
port = 40308
set_ssh_config(port) 

/app/data/.ssh/config file updated.


In [ ]:
from jupyter_client.kernelspec import KernelSpecManager
import subprocess

def register_remote_kernel(
    kernel_name="ipyf_remote_kernel",  # Any old name will do. This is fine.
    display_name="Remote Python",      # This is just what you'll see when you look at a list.
    remote_python="/path/to/python",  # Full path of Python executable to run on remote system.
    ssh_host_alias="remote_server_sshpyk", # Same alias as was used in writing to ssh config file.
    remote_kernel_name="python3",      # Typical for Jupiter.
    language="python",                 # Probably want to leave this unless you want to try R.
    verbose=True                       # Print extra info.
    ):
    ksm = KernelSpecManager()
    registered_names = list(ksm.get_all_specs().keys())
    if kernel_name in registered_names: 
        if verbose: print(f"{kernel_name} is already a registered kernel") 
    else: 
        if verbose: print(f"{kernel_name} is not a registered kernel. We need to add it") 
        subprocess.run(["sshpyk", "add", "--kernel-name", kernel_name,
            "--display-name", display_name, "--remote-python", remote_python, "--ssh-host-alias", ssh_host_alias,
            "--remote-kernel-name", remote_kernel_name, "--language", language
        ])
        if verbose: print("Success.")


In [ ]:
register_remote_kernel(remote_python='/Users/shawley/exercises/solveit/.venv/bin/python')

In [ ]:
#| export
#| notest
_ipf_km, _ipf_kc = None, None            # "ipf" = "ipyfernel" ;-) 
def ipf_startup(kernel_name="ipyf_remote_kernel"):  
    "Start up the remote kernel"
    global _ipf_km, _ipf_kc 
    if _ipf_km is None and _ipf_kc is None: #only do this at startup
        _ipf_km = KernelManager(kernel_name=kernel_name)
        _ipf_km.start_kernel()
        _ipf_kc = _ipf_km.client()
        _ipf_kc.start_channels()
        _ipf_kc.wait_for_ready(timeout=30)
        print("Success: remote kernel started")
    else: 
        print("ipf_startup: already running")

In [ ]:
ipf_startup()

Success: remote kernel started


Uh oh, I broke something. the String used to say "chonk_remote" or something, But it looked like it wasn't being used anywhere, so I thought I could set it to whatever I want.  so i thought maybe "ipyf_remote_kernel" would work better. Uh where is the kernel actually getting named though? In other words, why does that particular name matter? I'm missing something.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Ah, good question! The `kernel_name` parameter in `KernelManager` isn't arbitrary — it refers to a **registered kernel spec** that Jupyter knows about.

Kernel specs live in specific directories (like `~/.local/share/jupyter/kernels/` or `/usr/share/jupyter/kernels/`). Each kernel has a folder named after it, containing a `kernel.json` file that tells Jupyter how to launch that kernel.

You can see what kernels are actually installed with:

```bash
!jupyter kernelspec list
```

Run that and you'll see which kernel names are valid. "chonk_remote" must have been installed as a kernel spec somewhere — that's why it worked. "ipyf_remote_kernel" doesn't exist (yet), hence the error.

In [ ]:
!jupyter kernelspec list

Available kernels:
  chonk_python    /app/data/.local/share/jupyter/kernels/chonk_python
  python3         /usr/local/share/jupyter/kernels/python3


Okay, wait a second. I guess I completely forgot about this step. We're going to need to include this as part of the instructions or as part of the automation.   
So I guess that was done via this part of the sshpyk instructions?  ...scrape this url: https://github.com/casangi/sshpyk

```markdown 
Add a remote kernel (replace values with your configuration):
sshpyk add --ssh-host-alias remote_server_sshpyk \
            --kernel-name ssh_remote_python3 \
            --display-name "Remote Python 3.10" \
            --remote-python /path/to/python/env \
            --remote-kernel-name python3 \
            --language python
```

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Exactly! That's the missing piece. The `sshpyk add` command is what **creates the kernel spec** (the folder with `kernel.json` inside it) that gets registered with Jupyter.

So when you ran `ipf_startup(kernel_name="ipyf_remote_kernel")`, Jupyter looked for a folder at something like:
```
~/.local/share/jupyter/kernels/ipyf_remote_kernel/kernel.json
```

...and it doesn't exist because you never ran `sshpyk add` with that name.

The `chonk_python` kernel you see listed was presumably created earlier with a command like:

```bash
sshpyk add --kernel-name chonk_python ...
```

So you have two options:

1. **Change your code back** to use `kernel_name="chonk_python"` (the one that exists)
2. **Create a new kernel spec** with `sshpyk add --kernel-name ipyf_remote_kernel ...`

Want me to grab the full sshpyk docs to see all the options for that `add` command?

Well, I think I'd like to incorporate this into the `ipf_startup` rouotine. Or at least define a new function that gets called from IPF Startup. And so it can check to see if the requested kernel is already defined, and then if not, to add one. yea?  And we could just call `sshpyk add ...etc` via `subprocess` maybe?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yep, that's a solid approach! A few things to think through:

1. **Checking if kernel exists**: You could parse the output of `jupyter kernelspec list`, or just check if the path exists directly (e.g., `~/.local/share/jupyter/kernels/{kernel_name}/kernel.json`)

2. **Creating via subprocess**: `subprocess.run(["sshpyk", "add", ...])` would work nicely

3. **Parameters**: The `sshpyk add` command needs several arguments — some of these you'd want to pass in (like `--remote-python`), others could have sensible defaults

Which approach do you prefer for the existence check — parsing the command output or checking the filesystem directly? And do you want this as a separate helper function that `ipf_startup` calls, or baked right into `ipf_startup` itself?

re. 1. Do we have to really use the file system? Can we just do something from within Python since we're already inside of we've already got Jupiter running here on some level? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good thinking! Yes — you're already importing from `jupyter_client`, and it has a `KernelSpecManager` that can list and check for kernels programmatically:

```python
from jupyter_client.kernelspec import KernelSpecManager

ksm = KernelSpecManager()
ksm.get_all_specs()  # returns a dict of all installed kernels
```

You could then just check if your desired kernel name is in that dict. Want to try it out?

In [ ]:
from jupyter_client.kernelspec import KernelSpecManager

ksm = KernelSpecManager()
ksm.get_all_specs()  # returns a dict of all installed kernels

{'chonk_python': {'resource_dir': '/app/data/.local/share/jupyter/kernels/chonk_python',
  'spec': {'argv': ['/usr/local/bin/python',
    '/usr/local/bin/sshpyk-kernel',
    '--SSHKernelApp.kernel_name=chonk_python',
    "--KernelManager.connection_file='{connection_file}'"],
   'env': {},
   'display_name': 'Python on Chonk',
   'language': 'python',
   'interrupt_mode': 'message',
   'metadata': {'kernel_provisioner': {'provisioner_name': 'sshpyk-provisioner',
     'config': {'ssh': None,
      'ssh_host_alias': 'remote_server_sshpyk',
      'remote_python': '/Users/shawley/exercises/solveit/.venv/bin/python',
      'remote_kernel_name': 'python3',
      'ssh_config': '/app/data/.ssh/config'}}}}},
 'python3': {'resource_dir': '/usr/local/share/jupyter/kernels/python3',
  'spec': {'argv': ['python',
    '-m',
    'ipykernel_launcher',
    '-f',
    '{connection_file}'],
   'env': {},
   'display_name': 'Python 3 (ipykernel)',
   'language': 'python',
   'interrupt_mode': 'signal',
   

In [ ]:
list(ksm.get_all_specs().keys())

['chonk_python', 'python3']

In [ ]:
from jupyter_client.kernelspec import KernelSpecManager
import subprocess

def register_remote_kernel(kernel_name="ipyf_remote_kernel", verbose=True): 
    ksm = KernelSpecManager()
    registered_names = list(ksm.get_all_specs().keys())
    if kernel_name in registered_names: 
        if verbose: print("It's already there") 
    else: 
        if verbose: print("It's not there. we need to add it") 
        #subprocess.run('sspyk', ['add'])  #... I forget how to call sub process.

In [ ]:
register_remote_kernel('chonk_python') 

It's already there


Okay, you can see what I'm getting at with `register_remote_kernel` So should we use subprocess.run or subprocess.popen? Or should we s set shell equals true? Or can you help me with this line? Let's just fill it in.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

`subprocess.run` is perfect for this — it's simpler and just waits for the command to finish. No need for `shell=True` since you're passing a list of arguments directly.

The syntax is:

```python
subprocess.run(["sshpyk", "add", "--kernel-name", kernel_name, "--ssh-host-alias", "remote_server_sshpyk", ...])
```

You'll need to include the other required `sshpyk add` flags too (like `--remote-python`, `--remote-kernel-name`, etc.). Want to add those as parameters to your function, or hardcode some sensible defaults for now?

Here's the exact command we ran previously when we set this up the first time And it worked great: 
```
!sshpyk add --ssh-host-alias remote_server_sshpyk \
            --kernel-name chonk_python \
            --display-name "Python on Chonk" \
            --remote-python /Users/shawley/exercises/solveit/.venv/bin/python \
            --remote-kernel-name python3 \
            --language python
```
Okay. So we need a kernel name, an alias, and we need to know exactly which remote Python executable the user plans on using.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Right! So you'd want to add those as parameters to your function. Something like:

```python
def register_remote_kernel(
    kernel_name="ipyf_remote_kernel",
    display_name="Remote Python",
    remote_python="/path/to/python",
    ssh_host_alias="remote_server_sshpyk",
    remote_kernel_name="python3",
    language="python",
    verbose=True
):
```

Then the subprocess call becomes:

```python
subprocess.run([
    "sshpyk", "add",
    "--kernel-name", kernel_name,
    "--display-name", display_name,
    "--remote-python", remote_python,
    "--ssh-host-alias", ssh_host_alias,
    "--remote-kernel-name", remote_kernel_name,
    "--language", language
])
```

Want to wire that up and test it?

In [ ]:
from jupyter_client.kernelspec import KernelSpecManager
import subprocess

def register_remote_kernel(
    kernel_name="ipyf_remote_kernel",  # Any old name will do. This is fine.
    display_name="Remote Python",      # This is just what you'll see when you look at a list.
    remote_python="/path/to/python",  # Full path of Python executable to run on remote system.
    ssh_host_alias="remote_server_sshpyk", # Same alias as was used in writing to ssh config file.
    remote_kernel_name="python3",      # Typical for Jupiter.
    language="python",                 # Probably want to leave this unless you want to try R.
    verbose=True                       # Print extra info.
    ):
    ksm = KernelSpecManager()
    registered_names = list(ksm.get_all_specs().keys())
    if kernel_name in registered_names: 
        if verbose: print(f"{kernel_name} is already a registered kernel") 
    else: 
        if verbose: print(f"{kernel_name} is not a registered kernel. We need to add it") 
        subprocess.run(["sshpyk", "add", "--kernel-name", kernel_name,
            "--display-name", display_name, "--remote-python", remote_python, "--ssh-host-alias", ssh_host_alias,
            "--remote-kernel-name", remote_kernel_name, "--language", language
        ])
        if verbose: print("Success.")
    


In [ ]:
ipf_startup()

NameError: name 'ipf_startup' is not defined

Yea? 

In [ ]:
#| export
#| notest
def _output_hook(
    msg,   #  Message obtained from remote execution
    ):
    "How to handle output from the remote kernel."
    mt = msg["msg_type"]
    content = msg.get("content", {})
    if mt == "stream":
        print(content["text"], end="", flush=True)
    elif mt == "error":
        print('\n'.join(content.get("traceback", [])))
    elif mt in ("display_data", "update_display_data"):
        data = content.get("data", {})
        if "image/png" in data:
            display(Image(base64.b64decode(data["image/png"])))
        elif "text/plain" in data:
            print(data["text/plain"])

In [ ]:
#| export
#| notest
def ipf_exec(
    code:str,           # Code to be executed
    verbose=False,      # Return details about remote execution.
    ):
    "Execute code on the remote kernel." 
    assert _ipf_kc is not None, "ipf_exec: need to run ipf_startup() first"
    result = _ipf_kc.execute_interactive(code=code, output_hook=_output_hook)
    _ipf_kc.last_result = result  # stash it for optional inspection later
    if verbose: return result

In [ ]:
code = """
import platform 
print(platform.system())
"""
ipf_exec(code)

In [ ]:
#| export
#| notest
def ipf_shutdown(verbose=True):
    "Terminates the remote kernel"
    global _ipf_km, _ipf_kc
    if verbose: print("Shutting down remote kernel") # Note: Could make say if remote kernel is not even running.
    try:
        if _ipf_kc is not None: _ipf_kc.stop_channels()
        if _ipf_km is not None: _ipf_km.shutdown_kernel(now=True)  # 'now=True' forces immediate shutdown
    except: pass  # Don't hang on errors
    _ipf_km, _ipf_kc = None, None

In [ ]:
ipf_shutdown()

# iPython Magics

In [ ]:
#| export
#| notest
def _execute_remotely(lines:list[str]):
    "Take commands from magics and send to ipf_exec"
    code = ''.join(lines)
    if 'get_ipython()' in code: return lines  # let solveit internals pass through
    # Make sure our magics execute locally
    if code.strip().startswith(('%local', '%%local', '%unset_remote', '%resume_remote', '%set_remote', '%set_sticky','%unset_sticky')):
        return lines
    return [f"ipf_exec({repr(code)})\n"]

In [ ]:
#| export
#| notest
@register_line_magic
def set_remote(line:str):
    """Setup connection to remote server, start remote server, and enable 'sticky' remote execution of code cells (even without magics).
    usage: %set_remote <port> [user]"""
    parts = line.split()
    port = int(parts[0]) if parts else 65445
    user = parts[1] if len(parts) > 1 else ""
    set_ssh_config(port, user=user) 
    try: 
        ipf_startup()
    except Exception as e: 
        print(f"Error starting up remote kernel: {e}") 
        return 

In [ ]:
%set_remote {port}

UsageError: Line magic function `%set_remote` not found.


In [ ]:
#| export
#| notest
@register_line_cell_magic
def remote(line, cell=None):
    "remote exeuction: works as %remote and as %%remote" 
    ipf_exec(cell if cell else line)

In [ ]:
%%remote 
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

NameError: name 'ipf_exec' is not defined

In [ ]:
#| export
#| notest
@register_line_cell_magic
def local(line, cell=None):
    "local execution: works as %local and as %%local"
    get_ipython().run_cell(cell if cell else line) 

In [ ]:
%%local 
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

UsageError: Cell magic `%%local` not found.


In [ ]:
#| export
#| notest
@register_line_magic
def unset_remote(_):
    "shutdown remote server"
    unset_sticky('')  # get rid of any input transformers (see below) 
    ipf_shutdown()

## 'Sticky'/'Seamless' Remote Excution 

via Input Transformers.  These can make cells set execute remotely by default.

**WARNINGS**: 
1. Solve it is not intended to work with people modifying input transformers So be wary. Nevertheless, this seems to work.
2. If they're commands that you definitely want to execute locally, maybe run `%unset_sticky` just to be sure first.


In [ ]:
#| export
#| notest
gip = get_ipython()

@register_line_magic
def set_sticky(_):
    "Makes code cells execute remotely, via input transformer"
    assert _ipf_kc is not None, "Need an active remote kernel connection" 
    for f in gip.input_transformers_cleanup[:]:   # gaurd against appending twice
        if getattr(f, '__name__', '') == '_execute_remotely':
            print("Already executing remotely") 
            return 
    gip.input_transformers_cleanup.append(_execute_remotely)
    print('Code cells will now execute remotely.')

In [ ]:
#| export
#| notest
@register_line_magic
def unset_sticky(_):
    "Un-sticks remote execution for code cells" 
    for f in gip.input_transformers_cleanup[:]:  
        if getattr(f, '__name__', '') == '_execute_remotely':
            gip.input_transformers_cleanup.remove(f)
    print("Code cells will now run locally.") 



In [ ]:
%unset_remote 
print()
%set_remote {port}
%set_sticky

Code cells will now run locally.


NameError: name 'ipf_shutdown' is not defined

In [ ]:
# remote execution
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from 909e93a3a71e


In [ ]:
%unset_sticky

Code cells will now run locally.


In [ ]:
# local execution
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from 909e93a3a71e


In [ ]:
#| hide

#import nbdev; nbdev.nbdev_export()
!nbdev_export --procs scrub_magics